# Individual Analysis &ndash; $\alpha$ Range & Energy Loss


In [ ]:
import numpy as np  # import the numpy library functions.
import scipy.constants as const # import physical constants
import matplotlib.pyplot as plt # plotting functions
import uncertainties as unc # Uncertainties package.  Good for simple error propagation
from pandas import read_csv, DataFrame
# directive below puts the plots in the notebook
%matplotlib inline

## Calibrate the MCA system

### Find the MCA zero point

Enter the peak positions from the analysis of the  **alpha_zeros_cal.mca** file into the list `Pulser_channels`.

In [ ]:
Pulser_amplitude = [8.00, 6.00, 4.00, 2.00, 1.00] # Arbitrary units (Note order from high to low)
Pulser_channels = []  # Locate peak centroids in calibration mca file

Then fit to a line, with `Pulser_channels` as "Y" and `Pulser_amplitude` as "X" and obtain the intercept.  Create a pair numbers that gives this point on the "Energy" vs. "Channels" plane.  ("Energy" in this case = 0.  Right?) 

### Create a two-point calibration function

Write a function that takes as its arguments the coordinates of two points and returns the slope and intercept of a line that passes through those two points.

In [ ]:
def two_point_cal(p1, p2):
    '''Returns slope, intercept in a dictionary for two calibration points.
       Assumes p1, p2 are binary tuples: (x1,y1), (x2,y2).
    '''


### Apply the function to the Energy calibration

Find the peak position and width of the 5.486 MeV peak from Am-241.  Then use this data point to obtain the coefficients for one of the energy calibrations.

Apply the calibration to get the peak width (FWHM) in keV.

## Calibrate the pressure measurement system

Repeat the above process to obtain calibration coefficients for the pressure system.  You should be able to use your two-point function.

The data for the zero pressure voltage and atmospheric pressure voltage along with the atmospheric pressure in torr, are recorded in the hand-written data set on the course website in the file **Additional_alpha_range_data.pdf**. Use these data to obtain coefficents to convert voltage to torr.

## Get the Data and Calibrate it

### Read in the data

#### Advice
Use the Pandas `read_csv()` function to read the data files. It is very easy to use and handles a variety of data types and structures.  See below.

In the example, `N2` is a Pandas "dataframe".  If it is the only statement in the last line, it will print a nice table.

You may obtain the arrays for each column by using the column label, e.g., `N2['Pressure (V)']` is the array of the first column.

In [ ]:
# Read in the data sets:

from pandas import read_csv, DataFrame

# Here is an example
N2 = read_csv('Nitrogen_data.csv')


In [ ]:
print('Nitrogen Data Set')
N2

### Apply the calibration methods.  

Remember, here you convert Pressure in volts to torr (mm Hg) and channels to energy (MeV). 
**Important: Remember to use the peak location at zero pressure in the data set to determine the energy of the 5.486 MeV channel**.

Advice: After working out the steps to apply the calibrations, wrap those into a **function** that you can call with each data set as an argument, and which returns the calibrated data.

## Data Analysis

### Convert the pressure to mass-thickness $\rho x$

Use the standard density constants ($\rho_0$) from Sternheim (see the instructions) to convert the pressure measurements to mass thickness measurements.  You will need the temperature on the day the data were taken and the measured distance between the source and detector.  These are found in the hand-written part of the data sets.  Note: the Ar and N<sub>2</sub> data sets have a higher rom temperature and slightly shorter source-detector distance, as they were taken in 2019.

### Calculate $\Delta w$ 

As described in the instructions, you want to calculate the change in width of the peaks due to the gas.  This is done by subtracting the width of the zero-pressure peak from the widths of the other peaks "in quadrature."

Do this by making a function to subtract one number from an array in quadrature. The function should return the modified array.  You can reuse this function later.

In [ ]:
def quad_subtraction(w_array,w_0):
    '''Subract w_0 from all elements of w_array in quadrature'''


### Make three graphs

Plot:
1. $E$ vs. $\rho x$ (Energy vs mass thickness)
2. $\Delta w$ vs. $\rho x$ (Change in peak width vs mass thickness). Should you use the first data point? Why or why not?
3. $R$ vs. $\rho x$ (Counting rate vs. mass thickness)

Do this for all gases, and plot the data from each on the same graph so that you can compare them.

Plot the data with points, not lines, e.g., `'ob'` for blue dots.

## Calculate the "Stopping Power" $-dE/d(\rho x)$

#### First . . .
Write a couple of functions: one that takes an ordered array and returns the midpoints between each successive element in the array, e.g.  calculates $X_{mid,i} = (x_{i+1}+x_i)/2$, and another that takes a pair of related arrays, `X` and `Y`, and calculates an array of "first differences", that is
$$ (dy/dx)_i = \frac{y_{i+i}-y_i}{x_{i+1}-x_i}$$
(Note that this requires the `X` array to be strictly increasing.)

#### Second. . .

Apply your functions to the $\rho x$ and $E$ arrays to obtain the first-differences array $\Delta E/\Delta(\rho x)$ and the values of $\rho x$ at the midpoints, and plot the results for all gases on the same graph.

### Stopping Power as function of $E$

Now plot the same $\Delta E/\Delta(\rho x)$ array as a function of the deposited energy $E$ instead of $\rho x$. You will need the midpoints array of $E$ first: use your function.

### The Bethe-Bloch Equation

The "Bethe-Bloch" equation (or a modified form of its earlier incarnation as derived by Niels Bohr) applies to this experiment. This equation, from Leo, p. 24, written in terms of $\beta$ and specialized to lower energy $\alpha$ particles ($z=2$) looks like:

$$-\left(\frac{1}{\rho_0}\right)\frac{dE}{dx}=K\frac{Z}{A}\frac{4}{\beta^2}\left(\ln\frac{W^2_\text{max}}{I^2} - 2\beta^2\right)$$

where
* $K$ = $[2\pi r_e^2m_ec^2 N_\text{a}]$ = 0.1535 MeV-cm<sup>2</sup>/mol
* $\beta=v/c$ = speed of incoming particle
* $W_\text{max}=(2m_ec^2\beta^2)/(1-\beta^2)$ for lower energy particles
* $I$ is the mean excitation potential (in MeV)
* $Z$ is atomic number of gas specie (unitless)
* $A$ is molecular weight of gas (g/mol)
* $m_ec^2$ = rest energy of electron, 0.5110034 MeV

In this formula, the energy $E$ is the *kinetic energy* of the alpha particle.  You can obtain $\beta$ from the form of the relativistic kinetic energy $E = (\gamma -1)Mc^2$ where $\gamma\equiv 1/\sqrt(1-\beta^2)$

Write a function that will produce the above equation for input parameters $E$, $Z/A$ and $I$.  Then use the equation to generate theoretical curves for the different gases used, based on the parameters provided in the instructions  taken from the paper by Sternheimer, et al. 

For fundamental constants, use the SciPy constants library.

In [ ]:
# Define the constants and function used in the Bethe-Bloch equation

mc2 = const.value(u'electron mass energy equivalent in MeV')
Mc2 = const.value(u'alpha particle mass energy equivalent in MeV')
r_e = const.value(u'classical electron radius')*100.0 # Put electron radius in cm
N_a = const.value(u'Avogadro constant')



## Optional: Fit to Bethe-Bloch

You can use your function for the Bethe-Bloch equation to fit the data by letting the parameter $I$ be adjustable.  **Important: You can only do this where the formula is valid which would be above 1 MeV.** You would need to fit to a subset of the data.